In [3]:
import pandas
import sqlite3
import uuid
import names
import random

# Create connection

In [4]:
conn = sqlite3.connect('test.db')

# Populate with fake data

In [5]:
def create_table(table_name:str,con:sqlite3.Connection):
    df = pandas.DataFrame({
        'id':[str(uuid.uuid1()) for _ in range(10000)],
        'full_name':[names.get_full_name() for _ in range(10000)],
        'company_id':[random.randint(0,100000000000) for _ in range(10000)],
    })
    df['first_name'] = df['full_name'].str.split(' ').str.get(0)
    df['last_name'] = df['full_name'].str.split(' ').str.get(1)
    df['email'] = df['full_name'].str.replace(' ','') + '@email.com'
    df['role'] = 'user'
    df['password'] = random.randint(0,100000000000)
    df.to_sql(name='Users',con=conn,if_exists='replace',index=False,dtype={'user_id':'TEXT PRIMARY KEY'})

# Methods

In [6]:
def get_sql(sql:str,con:sqlite3.Connection) -> pandas.DataFrame:
    return pandas.read_sql_query(sql=sql,con=con)

def insert_sql(table_name:str,dataframe:pandas.DataFrame,con:sqlite3.Connection) -> pandas.DataFrame:
    return dataframe.to_sql(name=table_name,con=con,if_exists='append',index=False)

def create_sql_table(table_name:str,columns:pandas.DataFrame,con:sqlite3.Connection) -> pandas.DataFrame:
    return columns.to_sql(name=table_name,con=con,if_exists='fail')

# Send to SQL

In [83]:
create_table('Users',con=conn)
df_1 = pandas.DataFrame({
    'id':[str(uuid.uuid1())],
    'full_name':[names.get_full_name()],
    'company_id':[random.randint(0,100000000000)],
})
df_1['first_name'] = df_1['full_name'].str.split(' ').str.get(0)
df_1['last_name'] = df_1['full_name'].str.split(' ').str.get(1)
df_1['email'] = df_1['full_name'].str.replace(' ','') + '@email.com'
df_1['password'] = random.randint(0,100000000000)
df_1['role'] = 'user'

df_1.to_sql(name='Users',con=conn,if_exists='append',index=False)
df_1

,id,full_name,company_id,first_name,last_name,email,password,role
0,e5d25a4a-5749-11ee-8b08-b00b9af721a9,Kenneth Scott,11801922817,Kenneth,Scott,KennethScott@email.com,66655641152,user


# Get from SQL

In [85]:
user = {
    'email': 'KennethScott@email.com',
    'password':66655641152
}

In [100]:
user.update({'user_id':get_sql(f"""
    SELECT id FROM Users WHERE email = '{user['email']}' and password = '{user['password']}'
""")}) 

'e5d25a4a-5749-11ee-8b08-b00b9af721a9'

In [102]:
sql_data = pandas.read_sql_query(f"""
SELECT * From Users WHERE id = '{user['user_id']}' 
""",con=conn)

,id,full_name,company_id,first_name,last_name,email,role,password
0,e5d25a4a-5749-11ee-8b08-b00b9af721a9,Kenneth Scott,11801922817,Kenneth,Scott,KennethScott@email.com,user,66655641152
